# Memória, Variáveis, Comunicação e Estado de um Programa

Há várias possíveis estratégias para usar a memória de um computador para se realizar a comunicação entre partes dele -- por exemplo, entre duas funções -- e para se manter o **estado atual** de execução de um programa. Dentre elas, podemos destacar o uso de variáveis locais, globais e parâmetros entre funções.

Vamos fazer aqui um exercício nesse contexto e você será conduzido a experimentar cada uma dessas estratégias, mesmo que em algumas ocasiões não seja a forma recomendada de se desenvolver um programa.


# Exercício do Empréstimo

Em um financiamento com juros compostos e número de parcelas fixas parte-se dos seguintes parâmetros:
* `S` - valor da primeira parcela
* `N` - número de parcelas
* `J` - percentual de juros mensal

A primeira parcela a ser paga do financiamento é sempre igual a `S`. A partir daí é feita uma atualização mensal da parcela, em que cada nova parcela é calculada a partir da parcela do mês anterior, conforme a fórmula:

> Parcela<sub>mês</sub> = Parcela<sub>mês-1</sub> * (1 + `J` / 100)

O financiamento encerra quando as `N` parcelas são pagas.

Exemplo:
* `S`: 200
* `N`: 5
* `J`: 1%

Parcelas do financiamento:
`200`; `202`; `204.02`; `206.06`; `208.12`

## Exercício Parte 1 - Escrevendo um Programa

Dado o problema descrito, escreva um programa que calcule as parcelas de um empréstimo para os seguintes valores:
* `S`: 200
* `N`: 5
* `J`: 1%

Esses valores podem ser atribuídos na função `main`, não é necessária leitura pelo teclado. Entretanto, o programa deve ser genérico, ou seja, funcionará para qualquer outro valor atribuído às 3 variáveis.

In [3]:
#include <stdio.h>

int main() {
    int i, N=5;
    float S=200, J=1, parcela_anterior=S, parcela;
    
    printf("Parcela 1: %.2f\n", S);
    for(i = 2; i <= N; i++) {
        parcela = parcela_anterior * (1 + J/100);
        printf("Parcela %d: %.2f\n", i, parcela);
        parcela_anterior = parcela;
    }    
    return 0;
}

Parcela 1: 200.00
Parcela 2: 202.00
Parcela 3: 204.02
Parcela 4: 206.06
Parcela 5: 208.12


## Exercício Parte 2 - Escrevendo um Módulo (Função)

Reescreva o código acima de forma que seu programa faça uso de uma função `proximaParcela` que seja responsável pelo cálculo de cada parcela X do empréstimo.

Utilize as boas práticas de modularização que você aprendeu, evitando dependências do módulo com o programa em que está inserida. Considere que esta função poderia ser eventualmente reusada em outro programa e deveria ser possível fazê-lo sem que ela dependa de nada do programa em que irá se inserir.

A função deve apenas calcular uma única parcela em cada chamada, portanto, ficará a cargo da função principal que chama `proximaParcela` realizar o restante do processo, para que sejam apresentadas todas as parcelas do financiamento.

In [4]:
#include <stdio.h>

float proximaParcela(float parcela_anterior, float J, int primeira) {
    if (primeira)
        return parcela_anterior;
    return parcela_anterior * (1 + J/100);
}

int main() {
    int i, N=5;
    float S=200, J=1, parcela_anterior=S, proxima_parcela;
    
    printf("Parcela 1: %.2f\n", proximaParcela(parcela_anterior, J, 1));
    for(i = 2; i <= N; i++) {
        proxima_parcela = proximaParcela(parcela_anterior, J, 0);
        printf("Parcela %d: %.2f\n", i, proxima_parcela);
        parcela_anterior = proxima_parcela;
    }    
     
    return 0;
}

Parcela 1: 200.00
Parcela 2: 202.00
Parcela 3: 204.02
Parcela 4: 206.06
Parcela 5: 208.12


## Exercício Parte 3 - Minimizando os Parâmetros

Na solução anterior, você deve ter usado vários parâmetros para a comunicação entre o `main` com a função `proximaParcela`, evitando usar variáveis globais. Modifique a forma como você representa o empréstimo, de forma que a função `proximaParcela` receba sempre um único parâmetro capaz de representar todos os valores que estejam relacionados a um empréstimo.

In [4]:
#include <stdio.h>

float proximaParcela(float dados[]) {
    if (dados[2] == 1) { //primeira parcela
        dados[2] = 0; 
        return dados[1];
    }
    return dados[1] * (1 + dados[0]/100);
}

int main() {
    int i, N=5;
    float S=200, dados[3], proxima_parcela, J=1;
    
    dados[0] = J; //juros
    dados[1] = S; //parcela inicial
    dados[2] = 1; //para representar que eh a primeira parcela
    
    printf("Parcela 1: %.2f\n", proximaParcela(dados));
    for(i = 2; i <= N; i++) {
        proxima_parcela = proximaParcela(dados);
        printf("Parcela %d: %.2f\n", i, proxima_parcela);
        dados[1] = proxima_parcela;
    } 
    
    return 0;
}

Parcela 1: 200.00
Parcela 2: 202.00
Parcela 3: 204.02
Parcela 4: 206.06
Parcela 5: 208.12


## Exercício Parte 4 - Mantendo o Estado

A função que você codificou na solução anterior provavelmente não é capaz de manter o controle do que chamaremos de **estado do empréstimo**, ou seja, dados de qual a parcela corrente ou da última parcela calculada.

Modifique a função (módulo) `proximaParcela` acima de modo que ela seja capaz de guardar o **estado do empréstimo** por sua conta, sem que o `main` precise ficar informando este estado por parâmetros (por exemplo). Para isso serão permitidas as seguintes modificações:

1. você pode desmembrar a função em mais de uma, por exemplo, uma você chama no começo do empréstimo e outra a cada parcela;

2. você pode usar técnicas de uso de variáveis não recomendadas que geram dependência das funções com o programa em que ela está inserida.

Você deve organizar o código de tal maneira que o `main` informe para as funções os dados do financiamento apenas uma única vez e depois possa solicitar o cálculo da parcela subsequente sem informar tudo novamente. O ideal é que o main não tenha acesso ao modo como as funções vão representar o **estado do empréstimo**, para que toda a lógica do empréstimo fique dentro das funções.

In [5]:
#include <stdio.h>
#include <stdlib.h>

float J = 1;

typedef struct No {
    int num_parcela;
    float valor;
    struct No *prox; //o prox representa a parcela do mes anterior 
} No;

typedef struct No * p_no;

p_no criar_lista() {
    return NULL;
}

p_no adiciona_primeira_parcela(p_no lista, float valor) {
    p_no novo;
    novo = malloc(sizeof(No));
    if (novo == NULL) //verificando se há memoria suficiente
        exit(1);
    novo->num_parcela = 1;
    novo->valor = valor;
    novo->prox = lista;
    printf("Parcela %d: %.2f\n", novo->num_parcela, novo->valor);
    return novo;
}

float proximaParcela(p_no lista) {
    return lista->valor * (1 + J/100);
}

p_no adiciona_parcela(p_no lista) {
    p_no novo;
    novo = malloc(sizeof(No));
    if (novo == NULL) //verificando se há memoria suficiente
        exit(1);
    novo->num_parcela = lista->num_parcela + 1;
    novo->valor = proximaParcela(lista);
    novo->prox = lista;
    printf("Parcela %d: %.2f\n", novo->num_parcela, novo->valor);
    return novo;
}

void destruir_lista(p_no lista) {
    if (lista != NULL) {
        destruir_lista(lista->prox);
        free(lista);
    }
}

int main() {
    int i, S=200, N=5;
    p_no parcelas;

    parcelas = criar_lista();
    parcelas = adiciona_primeira_parcela(parcelas, S);

    for(i = 2; i <= N; i++) 
        parcelas = adiciona_parcela(parcelas);
    
    destruir_lista(parcelas);

    return 0;
}

Parcela 1: 200.00
Parcela 2: 202.00
Parcela 3: 204.02
Parcela 4: 206.06
Parcela 5: 208.12


## Exercício Parte 5 - Múltiplos Empréstimos

Considere que há múltiplos empréstimos que podem ser controlados em paralelo. A sua função `main` deve ser capaz de apresentar no console as parcelas de mais de um empréstimo de modo paralelo, mantendo um controle para cada **estado de empréstimo** separadamente. Você deve decidir qual das soluções tomará como ponto de partida, se deve modificar a função `main`, as funções de cálculo do empréstimo ou ambas para atender esse requisito da melhor forma possível.

Adote uma solução compacta e generalizável, de tal modo que comporte até 100 empréstimo e, cada novo empréstimo, só exija a informação dos parâmetros de partida, sem expansão do código.

Por exemplo, suponha os seguintes dois empréstimos em paralelo:

### Empréstimo 1
* `S`: 200
* `N`: 5
* `J`: 1%

### Empréstimo 2
* `S`: 500
* `N`: 7
* `J`: 2%

A saída esperada é:
~~~
Emprestimo 1: parcela 1 eh 200.00
Emprestimo 2: parcela 1 eh 500.00
Emprestimo 1: parcela 2 eh 202.00
Emprestimo 2: parcela 2 eh 510.00
Emprestimo 1: parcela 3 eh 204.02
Emprestimo 2: parcela 3 eh 520.20
Emprestimo 1: parcela 4 eh 206.06
Emprestimo 2: parcela 4 eh 530.60
Emprestimo 1: parcela 5 eh 208.12
Emprestimo 2: parcela 5 eh 541.22
Emprestimo 2: parcela 6 eh 552.04
Emprestimo 2: parcela 7 eh 563.08
~~~

O exemplo mostra dois empréstimos, mas a estrutura deve ser genérica o suficiente para controlar N empréstimos em paralelo (até o limite de 100).

O seu programa deve apresentar o exemplo acima como teste.

In [7]:
#include <stdio.h>
#include <stdlib.h>

typedef struct Parcelas {
    int num_parcela;
    float valor;
    struct Parcelas *prox; //o prox representa a parcela do mes anterior 
} Parcelas;

typedef struct Parcelas * p_no;

typedef struct {
    int num_emprestimo;
    p_no parcelas;
    int N;
    float J, S;
} Emprestimo;

float proximaParcela(Emprestimo emprestimo) {
    p_no lista = emprestimo.parcelas;
    return (lista->valor * (1 + emprestimo.J/100));
}

p_no adiciona_parcela(Emprestimo emprestimo, int primeira) {
    p_no lista = emprestimo.parcelas;
    p_no novo;
    novo = malloc(sizeof(Parcelas));
    if (novo == NULL) //verificando se há memoria suficiente
        exit(1);
    if (primeira) { //verificando se eh a primeira parcela
        novo->num_parcela = 1;
        novo->valor = emprestimo.S;
    } else {
        novo->num_parcela = lista->num_parcela + 1;
        novo->valor = proximaParcela(emprestimo);
    }
    novo->prox = lista;
    printf("Emprestimo %d: parcela %d eh %.2f\n", emprestimo.num_emprestimo, novo->num_parcela, novo->valor);
    return novo;
}

Emprestimo adiciona_emprestimo(int N, float J, float S, int num) {
    Emprestimo novo;
    novo.num_emprestimo = num;
    novo.parcelas = NULL;
    novo.N = N;
    novo.J = J;
    novo.S = S;
    return novo;
}

void destruir_lista(p_no lista) {
    if (lista != NULL) {
        destruir_lista(lista->prox);
        free(lista);
    }
}

int main() {
    int i=2;
    Emprestimo emprestimo1=adiciona_emprestimo(5, 1, 200, 1), emprestimo2=adiciona_emprestimo(7, 2, 500, 2);

    emprestimo1.parcelas = adiciona_parcela(emprestimo1, 1);
    emprestimo2.parcelas = adiciona_parcela(emprestimo2, 1);

    while(emprestimo1.N >= i || emprestimo2.N >= i) {
        if(emprestimo1.N >= i)
            emprestimo1.parcelas = adiciona_parcela(emprestimo1, 0);
        if(emprestimo2.N >= i)
            emprestimo2.parcelas = adiciona_parcela(emprestimo2, 0);
        i++;    
    }
    
    destruir_lista(emprestimo1.parcelas);
    destruir_lista(emprestimo2.parcelas);
    
    return 0;
}

Emprestimo 1: parcela 1 eh 200.00
Emprestimo 2: parcela 1 eh 500.00
Emprestimo 1: parcela 2 eh 202.00
Emprestimo 2: parcela 2 eh 510.00
Emprestimo 1: parcela 3 eh 204.02
Emprestimo 2: parcela 3 eh 520.20
Emprestimo 1: parcela 4 eh 206.06
Emprestimo 2: parcela 4 eh 530.60
Emprestimo 1: parcela 5 eh 208.12
Emprestimo 2: parcela 5 eh 541.22
Emprestimo 2: parcela 6 eh 552.04
Emprestimo 2: parcela 7 eh 563.08
